<a href="https://colab.research.google.com/github/miguelsmachado/airbnb_la_analysis/blob/master/An%C3%A1lise_de_M%C3%ADdias_Sociais_(Twitter).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Análise de Mídias Sociais**
*by [Miguel S. Machado](https://www.linkedin.com/in/miguel-machado-6655511a6/)*

---

# Analisando dados do Twitter - **Covid-19**

O estudo a abaixo tem como objetivo mostrar a relevância de um diagrama de relacionamentos para compressão do cenário atual.

Vivemos em um mundo onde a tecnologia está em constante desenvolvimento e  presente em todas as atividades que desempenhamos. Com essa evoulução, em um curto período de tempo as pessoas deixaram de ser simples consumidores passivos de informação e começaram a ganhar voz!
Hoje qualquer pessoa com um celular na mão pode se tornar um produtor de conteúdo e influenciador de pessoas pela internet.
Entender como as pessoas se posiocionam sobre um determinado assunto e quem são os principais ativistas desse assunto é muito importate. Se você é um jornalista, entender um diagrama de relacionamentos irá lhe proporcionar uma visão do todo e não só de um lado da moeda. Se você é um empresario, pode ver quem são os mais ativos e maiores influenciadores em assuntos associados a sua marca. Os usos desse tipo de informação são ilimitados.

Chega de conversa e vamos colocar as mãos na massa e entender o diagrama de relacionamentos sobre um dos principais assuntos do momento, infelizmente, o COVID-19.

Para realizar esse estudo serão utilizados 3 ferramentas:
- A primeira delas é o [FacePager](https://https://github.com/strohne/Facepager), que nos possibilitará a busca de dados no Twitter e criação do nosso arquivo quer irá gerar o nosso DataFrame;
- A segunda é a nossa poderosa linguagem de programação: Python. Com algumas bibliotecas fundamentais como Pandas, Matplotlib e Seaborn;
- E por último o [Gephi](https://https://gephi.org/users/download/), que será o grande responsável pela criação do diagrama de relacionamentos.


#**Entendendo o FacePager**

## Definição
O FacePager é uma ferramenta OpenSource que possibilita uma busca automatizada em algumas mídias sociais, como Twitter, Facebook e YouTube.

No nosso estudo, faremos um busca no Twitter. E para isso precisamos de uma conta para fazer o login (1). Feito o login, temos que criar um database (2). Se você nem sabe o que é, não se preocupe, não irá ultizar! O próximo passo é escolher os *Nodes* (3), que são nada mais nada menos que as palavras chaves. Agora vamos ajustar os padrões de busca (4), e quando tudo estiver conforme o desejado, selecione todos os nós e clique em `Fetch Data`.

Agora basta aguardar que os dados sejam coletados para que possamos iniciar o processo de exportação do arquivo *.csv*.

Antes de exportarmos, vamos selecionar as colunas que nos importam (5). E, finalmente, selecionar todos os nós e exportar o arquivo *.csv*.

#**Filtrando os dados com _Python_** 

## Importando as bibliotecas
Nosso primeiro passo é importar e instalar as bibliotecas que iremos trabalhar

In [0]:
!pip install wordcloud -q

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

## Criando o DataFrame e editando as variáveis

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/miguelsmachado/airbnb_la_analysis/master/covid19.csv", sep=';')

####Visualizando as 5 primeiras linhas do DataFrame

In [6]:
df.head()

,level,id,parent_id,object_id,object_type,query_status,query_time,query_type,user.screen_name,entities.user_mentions.0.screen_name,id.1,created_at,text,retweeted_status.favorite_count,retweeted_status.retweet_count,user.location,retweeted_status.user.location
0,0,1,None,covid-19,seed,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,1,1268357934691516417,data,fetched (200),2020-06-03 22:44:33.196458,Twitter:/search/tweets,camiperdao,brunogagliasso,1.268358e+18,Thu Jun 04 01:44:31 +0000 2020,RT @brunogagliasso: Já que o Ministério da Saú...,1945.0,514.0,Br,Brasil
2,1,3,1,1268357934171422720,data,fetched (200),2020-06-03 22:44:33.196458,Twitter:/search/tweets,mhelenaRS,DCM_online,1.268358e+18,Thu Jun 04 01:44:31 +0000 2020,RT @DCM_online: Freixo acusa ministro da Saúde...,15.0,7.0,"Rio Grande do Sul, Brasil","Sao Paulo, Brazil"
3,1,4,1,1268357932955172866,data,fetched (200),2020-06-03 22:44:33.196458,Twitter:/search/tweets,NeoJoao,Sarahteofilo,1.268358e+18,Thu Jun 04 01:44:30 +0000 2020,"RT @Sarahteofilo: Boletim 3/6 Em 24 horas, for...",2.0,3.0,Goiânia Rock City,Brasília
4,1,5,1,1268357929754857472,data,fetched (200),2020-06-03 22:44:33.196458,Twitter:/search/tweets,jvieiraleonardo,brunogagliasso,1.268358e+18,Thu Jun 04 01:44:30 +0000 2020,RT @brunogagliasso: Já que o Ministério da Saú...,1945.0,514.0,NaN,Brasil


#### Filtrando os dados para o Gephi

In [0]:
# De todas as colunas que recebemos no CSV original, só algumas nos interessam
# O primeiro passo é filtra-las.
df = df[['user.screen_name', 'entities.user_mentions.0.screen_name', 'object_id',
         'created_at', 'text', 'retweeted_status.retweet_count', 'user.location']]

# Agora vamos renomear as colunas para que fiquem mais fácil para nosso trabalho
# e entendimento do Gephi
df.columns = ['source', 'target', 'object_id', 'created_at', 'tweet_text', 
              'retweet_count', 'user.location']

# Como nosso estudo é focado em relacionamentos, não faz sentido com dados que 
# estão desconectados, ou seja, que não retwittes. As postagens que não são 
# retwittes recebem um valor NaN em sua coluna do retweet_count, que podem 
# facilmente ser excluidas com o comando abaixo
df = df.dropna()

#### Visualizando as 5 primeiras linhas do dado filtrado

In [16]:
print(f"O DataFrame possui {df.shape[0]} twetts")
print()
df.head()


O DataFrame possui 9882 twetts



,source,target,object_id,created_at,tweet_text,retweet_count,user.location
1,camiperdao,brunogagliasso,1268357934691516417,Thu Jun 04 01:44:31 +0000 2020,RT @brunogagliasso: Já que o Ministério da Saú...,514.0,Br
2,mhelenaRS,DCM_online,1268357934171422720,Thu Jun 04 01:44:31 +0000 2020,RT @DCM_online: Freixo acusa ministro da Saúde...,7.0,"Rio Grande do Sul, Brasil"
3,NeoJoao,Sarahteofilo,1268357932955172866,Thu Jun 04 01:44:30 +0000 2020,"RT @Sarahteofilo: Boletim 3/6 Em 24 horas, for...",3.0,Goiânia Rock City
5,cbxxss,WendelBrasilSP,1268357929587167234,Thu Jun 04 01:44:30 +0000 2020,RT @WendelBrasilSP: O q mídia progressista não...,28.0,Brasil
6,claudia20653209,RubensNunesMBL,1268357928702095366,Thu Jun 04 01:44:29 +0000 2020,RT @RubensNunesMBL: Infelizmente o Brasil queb...,2.0,"Niterói, Brasil"


#### Criando a coluna URL

Para que facilitar nossa analise, vamos criar uma nova coluna, manipulando alguns dados do nosso DataFrame e inserir o url do post que gerou o retweet.

In [0]:
for index, row in df.iterrows():
  df.loc[index, 'url'] = 'https://twitter.com/'+str(row['target'])+'/status/'+str(row['object_id'])

In [20]:
df.url.head()

1    https://twitter.com/brunogagliasso/status/1268...
2    https://twitter.com/DCM_online/status/12683579...
3    https://twitter.com/Sarahteofilo/status/126835...
5    https://twitter.com/WendelBrasilSP/status/1268...
6    https://twitter.com/RubensNunesMBL/status/1268...
Name: url, dtype: object

####Exportando o arquivo .csv

In [0]:
df.to_csv('data_gephi.csv', index=False)

##### **Com essa exportação, nosso arquivo esta pronto.**

Porém vamos utilizar algumas ferramentas que o python nos oferece para ver se existe algo que possa ser adicionado a busca do FacePager de forma a enriquecer nosso DataFrame

##**Refinando os parâmetros de busca**

### Criando a lista das palavras mais utilizadas